# SiRF Scanner

Iterate through folders, looking for file reader issues caused by wrong extension, missing headers, bad checksums, etc.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [71]:
import os
import sys

import pprint
import traceback

import numpy as np

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [72]:
def scanFiles():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions')

    exceptions = {}
    sats = {}
    satMeans = {}
    rawSummaries = {}
    
    fileCount = 0
    
    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ('.sbn', '.sbp'):
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    reader.load()
                    
                    # Record field summary
                    for field in reader.rawData.dtype.names:
                        pass
                        rawMin = reader.rawData[field].min()
                        rawMax = reader.rawData[field].max()
                        
                        if field in rawSummaries:
                            if reader.device in rawSummaries[field]:
                                    if rawMin < rawSummaries[field][reader.device]['min']:
                                        rawSummaries[field][reader.device]['min'] = rawMin
                                    if rawMax > rawSummaries[field][reader.device]['max']:
                                        rawSummaries[field][reader.device]['max'] = rawMax
                            else:
                                rawSummaries[field][reader.device] = {}
                                rawSummaries[field][reader.device]['min'] = rawMin
                                rawSummaries[field][reader.device]['max'] = rawMax
                        else:
                            rawSummaries[field] = {}
                            rawSummaries[field][reader.device] = {}
                            rawSummaries[field][reader.device]['min'] = rawMin
                            rawSummaries[field][reader.device]['max'] = rawMax

                    for track in reader.tracks:
                        item = 'ehve'
                        if item in track.data:
                            # Record satellites
                            if reader.device in sats:
                                sats[reader.device].append(track.data[item])
                            else:
                                sats[reader.device] = [track.data[item]]

                            # Record mean
                            satMean = track.data[item].mean()
                            if reader.device in satMeans:
                                satMeans[reader.device].append(satMean)
                            else:
                                satMeans[reader.device] = [satMean]

                    print('.', end='')

                except Exception:
                    exceptions[file] = traceback.format_exc()
                    
                    print(traceback.format_exc())

                    print('E', end='')

                fileCount += 1

        if fileCount < 0:
            break
                
    pp = pprint.PrettyPrinter(indent=4, compact=False)

    # Show raw summaries
    #print(os.linesep * 2 + 'Summaries:')
    #for field in rawSummaries:
        #print(field)
        #pp.pprint(rawSummaries[field])

    # Show stats
    print(os.linesep * 2 + 'medians:')
    for device in sats:
        print(device)

        tmp = np.concatenate(sats[device], axis=0)
        print(np.median(tmp[np.nonzero(tmp)]))

        #tmp = tmp[np.where(tmp != 2.55)]
        #tmp = tmp[np.where(tmp != 0)]
        print(np.median(tmp))

        #unique, counts = np.unique(tmp, return_counts=True)
        #print(np.asarray((unique, counts)).T)

    # Report errors
    #print(os.linesep * 2 + 'Errors:')
    #if len(exceptions) > 0:
        #for file in exceptions:
            #print(os.linesep + file)
            #print(exceptions[file].split(os.linesep)[-2:-1])
    #else:
        #print(os.linesep + 'None!')

In [73]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    scanFiles()    

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................